## <center>MSM-building Calculations</center>

##### MUST SHARE KERNEL WITH NOTEBOOK `00-load-data.ipynb`

#### TICA and VAMP

------------------------

In [20]:
da = analysis_directory
if not da.is_dir():
    da.mkdir()

print(colorama.Back.CYAN+"Starting MSM Pipeline with TICA")
print("Creating an array of models")

# uses last for `nm`,`feat` from kernel connection!!!
n_ticas = len(all_models[feat][nm]["tica"])

for n,(feat,nm,dataset) in enumerate(
    aswa_tools.iter_models(all_models)
):
    dds = da / nm
    dtica = dds / "tica"
    ftica = dtica / "results.pyemma"
    dvamp = dds / "vamp"
    fvamp = dvamp / "results.pyemma"
    
    if not dds.is_dir():
        dds.mkdir()
    if not dtica.is_dir():
        dtica.mkdir()
    if not dvamp.is_dir():
        dvamp.mkdir()

    for m,setup in enumerate(dataset["tica"]):
        
        thisone = _model_name(feat, setup["kwargs"])
        
        #clear_output(wait=True)
        print(colorama.Back.CYAN+colorama.Fore.BLACK+\
            "Calculated %d models so far"%(n_ticas*n+m))
        print((" - on to model %s"%thisone)+\
            colorama.Back.RESET)
        print(colorama.Fore.LIGHTGREEN_EX+\
            " - number %d / %d (wrong 2nd number)"%(
            n_ticas*n+m,n_ticas*len(all_models)))
        
        _ftica = ftica.with_suffix(".%s"%thisone)
        
        if _ftica.is_file():
            print("Found Results File: {}".format(_ftica))
            setup["result"] = pyemma.load(_ftica)
            
        else:
            print(
    "Calculating and Saving to File: {}".format(_ftica))
            
            try:
                setup["result"] = coor.tica(
                    setup["input"], **setup["kwargs"])
                setup["result"].save(
                    _ftica, save_streaming_chain=True)

            except:
                print(
    colorama.Back.YELLOW+"Calculation failed!")
                print(
    "Saving the error in place of model: SOON! FIXME")
                print(
    colorama.Back.RESET+colorama.Fore.LIGHTRED_EX+\
                    traceback.format_exc())

#    for setup in dataset["vamp"]:
 #       _fvamp = fvamp.with_suffix(
  #          ".%s"%_model_name(feat, setup["kwargs"]))
   #     
    #    if _fvamp.is_file():
     #       print("Found Results File: {}".format(_fvamp))
      #      setup["result"] = pyemma.load(_fvamp)
       #     
        #else:
         #   print("Calculating and Saving to File: {}".format(_fvamp))
          #  setup["result"] = coor.vamp(setup["input"], **setup["kwargs"])
           # setup["result"].save(_fvamp, save_streaming_chain=True)

print(colorama.Back.LIGHTBLACK_EX + \
      colorama.Fore.BLUE + \
      "  DONE LOADING DATA  ")

Starting MSM Pipeline with TICA
Creating an array of models
Calculated 0 models so far
 - on to model invca__lag-500__dim-20
 - number 0 / 2 (wrong 2nd number)
Found Results File: analyses/ltj/tica/results.invca__lag-500__dim-20
Calculated 1 models so far
 - on to model invca__lag-1000__dim-20
 - number 1 / 2 (wrong 2nd number)
Found Results File: analyses/ltj/tica/results.invca__lag-1000__dim-20
Calculated 2 models so far
 - on to model invca__lag-500__dim-20__weights-koopman
 - number 2 / 2 (wrong 2nd number)
Found Results File: analyses/xma_Ca/tica/results.invca__lag-500__dim-20__weights-koopman
Calculated 3 models so far
 - on to model invca__lag-1000__dim-20__weights-koopman
 - number 3 / 2 (wrong 2nd number)
Calculating and Saving to File: analyses/xma_Ca/tica/results.invca__lag-1000__dim-20__weights-koopman
Calculation failed!
Saving the error in place of model: SOON! FIXME
Traceback (most recent call last):
  File "<ipython-input-20-c556d1368d22>", line 52, in <module>
    setu

#### k-means and regular-space clusterings
------------------------

In [21]:
# uses last for `nm`,`feat` from kernel connection!!!
n_kmeans    = len(all_models[feat][nm]["cluster_kmeans"])
n_regspaces = len(all_models[feat][nm]["cluster_regspace"])

for n,(feat,nm,dataset) in enumerate(
    aswa_tools.iter_models(all_models)
):
    dds       = da / nm
    dkmeans   = dds / "kmeans" / ("tica_lag_%d"%chosen_lag)
    dregspace = dds / "regspace" / ("tica_lag_%d"%chosen_lag)
    fkmeans   = dkmeans / "results.pyemma"
    fregspace = dregspace / "results.pyemma"

    if not dkmeans.is_dir():
        dkmeans.mkdir(parents=True)

    if not dregspace.is_dir():
        dregspace.mkdir(parents=True)
    
    print(colorama.Back.CYAN+\
        "Next is k-means clustering TICA")

    for m,setup in enumerate(dataset["cluster_kmeans"]):
        
        thisone = _model_name(feat, setup["kwargs"])
        
        print(colorama.Back.CYAN+\
            "Next is k-means clustering TICA")
        print(colorama.Back.CYAN+\
            "Calculated %d models so far"%(n_kmeans*n+m))
        print((" - on to model %s"%thisone)+\
            colorama.Back.RESET)
        print(colorama.Fore.LIGHTGREEN_EX+\
            " - number %d / %d (wrong 2nd number)"%(
            n_kmeans*n+m,(n_kmeans+n_regspaces)*len(
                all_models)))

        _fkmeans = fkmeans.with_suffix(".%s"%thisone)

        setup["input"] = aswa_tools.get_matching_input(
            dataset["tica"], key="lag", val=chosen_lag)

        if _fkmeans.is_file():
            print("Found Results File: {}".format(_fkmeans))
            setup["result"] = pyemma.load(_fkmeans)
            
        else:
            print(
    "Calculating and Saving to File: {}".format(_fkmeans))

            if not setup["input"]:
                continue

            # SUM TING WONG with TICA object
            setup["input"]._default_chunksize = 10000

            tica_inp = setup["input"].get_output(
                list(range(n_tica_dim)))
            
            print(colorama.Fore.BLACK+colorama.Back.BLUE+\
                  pformat(len(tica_inp)))
            print([pformat(ti.shape) for ti in tica_inp])
            
            setup["result"] = coor.cluster_kmeans(
                tica_inp, **setup["kwargs"])

            setup["result"].save(
                _fkmeans, save_streaming_chain=True)

    print(colorama.Back.CYAN+\
        "k-means done, now regspace clustering TICA")

    for m,setup in enumerate(
        dataset["cluster_regspace"]
    ):
        thisone = _model_name(feat, setup["kwargs"])
        
        #clear_output(wait=True)
        print(colorama.Back.CYAN+\
            "Next is regspace clustering TICA")
        print(colorama.Back.CYAN+\
            "Calculated %d models so far"%(
            n_kmeans*(1+n)+m+n_regspaces*n))
        print((" - on to model %s"%thisone)+\
            colorama.Back.RESET)
        print(colorama.Fore.LIGHTGREEN_EX+\
            " - number %d / %d (wrong 2nd number)"%(
            n_kmeans*n+m,(n_kmeans+n_regspaces)*len(
                all_models)))

        _fregspace = fregspace.with_suffix(".%s"%thisone)

        setup["input"] = aswa_tools.get_matching_input(
            dataset["tica"], key="lag", val=chosen_lag)
                
        if not setup["input"]:
            continue

        # SUM TING WONG with TICA object
        setup["input"]._default_chunksize = 10000

        tica_inp = setup["input"].get_output(
            list(range(n_tica_dim)))

        mindist = np.power(np.product([
            mx-mn for mn,mx in zip(
                np.min(np.concatenate(tica_inp), axis=0),
                np.max(np.concatenate(tica_inp), axis=0))
        ]), 1./n_tica_dim) / 25.
        
        setup["actual-dmin"] =setup["kwargs"]["dmin"]*mindist
            
        if _fregspace.is_file():
            print("Found Results File: {}".format(
                _fregspace))
            
            setup["result"] = pyemma.load(_fregspace)
            
        else:

            print("Calculating and Saving to File: {}".format(
                _fregspace))
            
            setup["result"] = aswa_tools.refined_regspace(
                tica_inp, mindist, **setup["kwargs"])
            
            setup["result"].save(
                _fregspace, save_streaming_chain=True)
            
            
print(colorama.Back.LIGHTBLACK_EX + \
      colorama.Fore.BLUE + \
      "  DONE LOADING DATA  ")

Next is k-means clustering TICA
Next is k-means clustering TICA
Calculated 0 models so far
 - on to model invca__k-75__max_iter-100__stride-5
 - number 0 / 3 (wrong 2nd number)
Found Results File: analyses/ltj/kmeans/tica_lag_1000/results.invca__k-75__max_iter-100__stride-5
Next is k-means clustering TICA
Calculated 1 models so far
 - on to model invca__k-300__max_iter-100__stride-5
 - number 1 / 3 (wrong 2nd number)
Found Results File: analyses/ltj/kmeans/tica_lag_1000/results.invca__k-300__max_iter-100__stride-5
Next is k-means clustering TICA
Calculated 2 models so far
 - on to model invca__k-600__max_iter-100__stride-5
 - number 2 / 3 (wrong 2nd number)
Found Results File: analyses/ltj/kmeans/tica_lag_1000/results.invca__k-600__max_iter-100__stride-5
k-means done, now regspace clustering TICA
Next is k-means clustering TICA
Next is k-means clustering TICA
Calculated 3 models so far
 - on to model invca__k-75__max_iter-100__stride-5
 - number 3 / 3 (wrong 2nd number)
Calculating and

#### Many MSMs calculated to examine Implied TimeScales
----------------------------------

In [25]:
for n,(feat,nm,dataset) in enumerate(
    aswa_tools.iter_models(all_models)
):  
    dataset["its"] = dlits = dict()

    for m,(clust_method, par_key) in enumerate(
        [("regspace", "dmin"), ("kmeans", "k")]
    ):    
        dlits[clust_method] = lits = list()
        
        dds = da / nm
        dmsm = dds / "msm" / ("tica_lag_%d"%chosen_lag)
        fits = dmsm / "its-results.pyemma"
        
        if not dmsm.is_dir():
            dmsm.mkdir(parents=True)
        
        for i,setup in enumerate(
            dataset["cluster_%s"%clust_method]
        ):
            thisone = _model_name(
                feat,
                reduce(lambda x,y: dict(x,**y), (
                    {"weights" : "empirical"
                     # OOM reweight seems absurdly slow...
                     #   if dataset["n_trajs"]==1
                    #    else "oom"
                    },
                    setup["kwargs"])))
            
            
            thisone = _model_name(feat,
                {"weights":"%s"%("empirical"
                    if dataset["n_trajs"]==1
                    else "oom")}.update(setup["kwargs"]))
                    
            #clear_output(wait=True)
            print(colorama.Back.CYAN+\
                "Now creating MSMs using clusterings"+\
                "as the discretization to state space")
            print(colorama.Back.CYAN+\
                "Calculated %d models so far"%(m))
            print((" - on to model %s"%thisone)+\
                colorama.Back.RESET)
            print(colorama.Fore.LIGHTGREEN_EX+\
                " - number %d / %d (wrong 2nd number)"%(
                n_kmeans*n+m,(n_kmeans+n_regspaces)*len(
                    all_models)))

            _fits = fits.with_suffix(
                ".%s-%s"%(clust_method, thisone))
            
            dits           = dict()
            
            dits["par"]    = (
                par_key, setup["kwargs"][par_key])
            
            dits["kwargs"] = dict(
                lags=tica_lags,
                reversible=False,
                nits=10,
                errors="bayes",
                weights="empirical" if dataset["n_trajs"]==1 else "oom",
            )
            
            if not _fits.is_file():
                if not setup["result"]: continue 

                print(
colorama.Back.BLUE+colorama.Fore.BLACK+"drajs shape:")
                print(
str(len(setup["result"].dtrajs)))
                print(pformat([dt.shape
for dt in setup["result"].dtrajs])+colorama.Back.RESET)
                print(colorama.Fore.LIGHTGREEN_EX+\
"Calculating and Saving to File: {}".format(_fits))
                
                pyemma.msm.its(
                #dits["result"] = pyemma.msm.its(
                    #dits["input"],
                    setup["result"].dtrajs,
                    nsamples=20,
                    **dits["kwargs"]
                ).save(_fits)
                dits["result"] = _fits
                #dits["result"].save(_fits)
            
            else:
                print("Found Results File: {}".format(_fits))
                dits["result"] = pyemma.load(_fits)
                #dits["result"] = _fits
            
            # Reverse so both with increasing N clusters
            if "regspace" in clust_method:
                lits.insert(0, dits)
            else:
                lits.append(dits)


print(colorama.Back.LIGHTBLACK_EX + \
      colorama.Fore.BLUE + \
      "  DONE LOADING DATA  ")

NameError: name 'reduce' is not defined

In [ ]:
print(colorama.Back.LIGHTBLACK_EX + \
      colorama.Fore.BLUE + \
      "  DONE LOADING DATA  ")

#### Many HMMs
------------------

 - Save a "failed" file and loook at it if results file not found, change results tag to failed or something

In [ ]:
reversible = False
howmany = 0
n_kmeans    = len(all_models[feat][nm]["cluster_kmeans"])
n_regspaces = len(all_models[feat][nm]["cluster_regspace"])

for feat,nm,dataset in aswa_tools.iter_models(all_models):

    dataset["hits"] = dlits = dict()

    for clust_method, par_key in [
        ("regspace", "dmin"), ("kmeans", "k")
    ]:
    #for clust_method, par_key in [("kmeans", "k")]:
    #for clust_method, par_key in [("regspace", "dmin")]:
    
        dlits[clust_method] = lits = list()

        dds = da / nm
        dmsm = dds / "hmm" / ("tica_lag_%d"%chosen_lag)
        fits = dmsm / "its-results.pyemma"

        if not dmsm.is_dir():
            dmsm.mkdir(parents=True)
   
        for n in n_macrostates:
            for i,setup in enumerate(
                dataset["cluster_%s"%clust_method]
            ):
                #clear_output(wait=True)
                howmany += 1
                
                thisone = "%s-%s_%d-%s_%s-%s"%(
                    clust_method, "nstates", n,
                    "reversible", str(reversible),
                    _model_name(feat, setup["kwargs"]))
                
                print(colorama.Back.CYAN + \
                    "Calculated %d models so far"%(howmany))
                print((" - on to model %s"%thisone) + \
                    colorama.Back.RESET)
                print(colorama.Fore.LIGHTGREEN_EX + \
    " - number %d / %d (wrong 2nd number)"%(howmany,
    (n_kmeans+n_regspaces)*n_macrostates[-1]*len(all_models)))

                _fits = fits.with_suffix(".%s"%thisone)

                print("fitted")
                dits           = dict()
                print("getting dtrajs")
                
                print("dtrajs were gotten")
                dits["par"]    = (
                    (par_key, setup["kwargs"][par_key]),
                    ("nstates", n))

                print("kwargs on it")
                dits["kwargs"] = dict(
                    lags=tica_lags,
                    nstates=n,
                    reversible=reversible,
                    #nits=3,
                    errors="bayes",
                    nsamples=20,
                )

                dits["result"] = False
                print("checking for existing files")
                
                if not _fits.is_file():
                    
                    print(
    colorama.Back.MAGENTA+"Calculating and Saving to File: {}".format(
        _fits)+colorama.Back.RESET)
                    
                    try:
     #                   ditsinput = setup["result"].dtrajs
                        dits["input"]  = setup["result"].dtrajs
        
                    except AttributeError:
                        print(
    colorama.Back.RED+"MODEL NOT FOUND"+colorama.Back.RESET)

                    try:
                        #pyemma.msm.timescales_hmsm(
                        dits["result"] = pyemma.msm.timescales_hmsm(
                            dits["input"],
                            **dits["kwargs"]
                        #)
                        #dits["result"].save(_fits)
                        ).save(_fits)
                        
                    except:
                        dits["result"] = False

                else:
                    print(
    colorama.Back.GREEN+colorama.Fore.RESET+"Found Results File: {}".format(
    _fits)+colorama.Back.RESET)
                    
                    try:
                        dits["result"] = pyemma.load(_fits)
                        
                    except:
                        dits["result"] = False
                        
                    #dits["result"] = _fits

                # Reverse so both with increasing N clusters
                if "regspace" in clust_method:
                    lits.insert(0, dits)
                    
                else:
                    lits.append(dits)
                    
                    
print(colorama.Back.LIGHTBLACK_EX + \
      colorama.Fore.BLUE + \
      "  DONE LOADING DATA  ")

-----------------

-----------------

#### $\infty$ DONE
-------

In [ ]:
print(colorama.Back.LIGHTBLACK_EX + \
      colorama.Fore.BLUE + \
      "  DONE LOADING DATA  ")

-----------------

-----------------

------------

------------

------------

-----------

-----------

-----------

-----------

-----------

------------